# Constructing a mapping

A mapping is written by an EO expert and contains rules that map semantic concepts formalized in an ontology to data values stored in an EO data cube. As such, the mapping forms the connection between the semantic domain and the image domain. 

This notebook describes the data structures to represent a mapping in semantique, and shows how an EO expert can construct a mapping from scratch.

## Content

- [Structure](#Structure)
- [The Semantique configuration](#The-Semantique-configuration)

## Prepare

Import packages:

In [1]:
import semantique as sq

## Structure

*To be added*

## The Semantique configuration

*To be added*